# Intro to LLMstudio


## start proxy server


In [1]:
from llmstudio.server import start_server
start_server()

Running LLMstudio Engine on http://0.0.0.0:8001 
Running LLMstudio Tracking on http://0.0.0.0:8002 


In [2]:
from llmstudio.engine.provider import LLMProxyProvider as LLM
from llmstudio.engine.provider import ProxyConfig

# from llmstudio_core import LLMCore as LLM
# from llmstudio import LLM

llm = LLM(provider="openai", 
          proxy_config=ProxyConfig(host="0.0.0.0", port="8001"))

Connected to LLMStudio Proxy @ 0.0.0.0:8001


In [3]:
result = llm.chat("olá", model="gpt-4o")


In [4]:
result.chat_output, result.metrics

('Olá! Como posso ajudar você hoje?',
 {'input_tokens': 2,
  'output_tokens': 11,
  'total_tokens': 13,
  'cost_usd': 0.000175,
  'latency_s': 0.6496210098266602,
  'time_to_first_token_s': 0.5341501235961914,
  'inter_token_latency_s': 0.01241710450914171,
  'tokens_per_second': 15.393590799454474})

### Import clients


In [5]:
from llmstudio import LLM
from pprint import pprint

In [6]:
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
from llmstudio.engine.provider import ProxyConfig
proxy = ProxyConfig(host="0.0.0.0", port="8001")
print(proxy)

openai = LLM("openai", proxy_config=proxy)


host='0.0.0.0' port='8001' url=None username=None password=None
Connected to LLMStudio Proxy @ 0.0.0.0:8001


In [7]:
openai._provider


### Chat (non-stream)

In [8]:
openai.chat("What's your name", model="gpt-4o")

ChatCompletion(id='7a435f11-ab93-473e-b439-37202e554a1f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I don’t have a personal name, but you can call me Assistant. How can I help you today?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729076083, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output='I don’t have a personal name, but you can call me Assistant. How can I help you today?', chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729076085.7050822, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 22, 'total_tokens': 26, 'cost_usd': 0.00035, 'latency_s': 3.0498709678649902, 'time_to_first_token_s': 0.5896170139312744, 'inter_token_latency_s': 0.10692943697390349, 'tokens_per_second': 7.86918536976690

#### Async version

In [9]:
await openai.achat("What's your name", model="gpt-4o")

ChatCompletion(id='a94bffc5-88ab-4290-8cce-d349c9829e57', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I’m called ChatGPT. How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729076085, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output='I’m called ChatGPT. How can I assist you today?', chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729076086.277256, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 14, 'total_tokens': 18, 'cost_usd': 0.00023, 'latency_s': 0.5512211322784424, 'time_to_first_token_s': 0.3975660800933838, 'inter_token_latency_s': 0.010929771832057409, 'tokens_per_second': 27.212309401126042})

### Chat (stream)

In [10]:
response = openai.chat("Write a paragfraph about space", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, often referred to as the cosmos, is an expansive and mysterious expanse that extends beyond

 Earth's atmosphere. It is a vast, seemingly infinite realm filled with celestial bodies such as stars, planets

, meteoroids, comets, and galaxies, each exhibiting unique characteristics and phenomena. The study

 of space, known as astronomy, has intrigued humans for millennia, driving them to explore and understand

 the universe's origins, structure, and future. With advancements in technology, humans have embarked on remarkable

 journeys into space, from landing on the Moon to deploying telescopes like the Hubble Space Telescope,

 which capture breathtaking images of distant galaxies and nebulas. Despite these achievements, the cosmos remains largely unexpl

ored and holds countless mysteries, fueling our curiosity about the possibility of extraterrestrial life, the nature of

 dark matter and dark energy, and the ultimate fate of the universe itself. The exploration of space co

#### Async version

In [11]:
i=0
async for chunk in await openai.achat("Write a paragfraph about space", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




Space, the vast expanse beyond our planet's atmosphere, is a realm of awe and mystery

 that has captivated humanity for centuries. It stretches infinitely, hosting a myriad of celestial bodies, including stars

, planets, moons, asteroids, and comets, all bound by the forces of gravity.

 The cosmos is not just a backdrop for astronomical phenomena; it's a dynamic environment where the laws of physics

 come to life. Black holes bend time and space, while supernovae illuminate the cosmos with explosive

 brilliance. Beyond our Milky Way galaxy, billions of other galaxies swirl in a cosmic dance, each

 potentially hosting its own unique systems and perhaps even life. As we continue to explore this enigmatic frontier through

 telescopes and space missions, we uncover the origins of the universe, the nature of dark matter,

 and the possibility of extraterrestrial life, igniting our imagination and expanding our understanding of existence itself.



## Metrics:
{'cost_usd': 0.0001098

#### Batch version

In [12]:
from llmstudio import LLM
import nest_asyncio
nest_asyncio.apply()

In [14]:
gpt3 = LLM('openai')

In [15]:
complex_prompt= [{"role": "system", "content": "You are a tchill dude from jamaica"},
          {"role": "user", "content": "Hello, how are you?"},
          {"role": "assistant", "content": "Ye man doin fain man!"},
          {"role": "user", "content": "How is the weather today?"}]

inputs = [
    "What's your name?",
    "Tell me a joke.",
    "What's the weather like?",
    "Can you sing a song?",
    "Tell me about yourself.",
    complex_prompt
]

responses = gpt3.run_batch_chat_coroutine(inputs)
responses

AttributeError: 'LLM' object has no attribute 'run_batch_chat_coroutine'